# Load in house sales data

In [32]:
import pandas as pd
import numpy as np
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [33]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

In [34]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

### Find what features had non-zero weight.

In [35]:
model_all.coef_

array([     0.        ,      0.        ,      0.        ,    134.43931396,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,  24750.00458561,      0.        ,
        61749.10309071,      0.        ,      0.        ,     -0.        ,
            0.        ])

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set.

In [36]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [37]:
training.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
1,7237550310,20140512T000000,1225000.0,4.0,4.50,5420.0,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760.0,101930.0
2,9212900260,20140527T000000,468000.0,2.0,1.00,1160.0,6000,1.0,0,0,...,7,860,300,1942,0,98115,47.6900,-122.292,1330.0,6000.0
3,0114101516,20140528T000000,310000.0,3.0,1.00,1430.0,19901,1.5,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780.0,12697.0
4,6054650070,20141007T000000,400000.0,3.0,1.75,1370.0,9680,1.0,0,0,...,7,1370,0,1977,0,98074,47.6127,-122.045,1370.0,10208.0


In [38]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [39]:
def get_RSS(prediction, output):
    residual = prediction - output
    RS = residual * residual
    RSS = sum(RS)
    return(RSS) 

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [40]:
for l1_penalty in np.logspace(1, 7, num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    prediction = model.predict(validation[all_features])
    RSS = get_RSS(prediction, validation['price'])
    print l1_penalty
    print RSS

10.0
3.982133273e+14
31.6227766017
3.99041900253e+14
100.0
4.29791604073e+14
316.227766017
4.63739831045e+14
1000.0
6.45898733634e+14
3162.27766017
1.22250685943e+15
10000.0
1.22250685943e+15
31622.7766017
1.22250685943e+15
100000.0
1.22250685943e+15
316227.766017
1.22250685943e+15
1000000.0
1.22250685943e+15
3162277.66017
1.22250685943e+15
10000000.0
1.22250685943e+15


# Best l1_penalty is 10

In [41]:
best_l1_penalty = 10
model = linear_model.Lasso(alpha=best_l1_penalty, normalize=True)
model.fit(training[all_features], training['price'])


Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [42]:
model.coef_   

array([ -1.61445628e+04,   3.73245384e+02,   5.08412433e+04,
         6.17853560e+02,  -4.44113549e+04,   7.85623065e-01,
        -7.01194765e+02,  -0.00000000e+00,   5.01420046e+03,
         6.19488752e+05,   3.80418557e+04,   2.49987718e+04,
         1.28716235e+05,   0.00000000e+00,   0.00000000e+00,
        -3.29383118e+03,   1.00573209e+01])

In [43]:
model.intercept_

6630155.6686283723

In [44]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

# Limit the number of nonzero weights

In [50]:
max_nonzeros = 7

# Exploring the larger range of values to find a narrow range with the desired sparsity

In [51]:
l1_penalty_values = np.logspace(1, 4, num=20)
list_nonzero =[]

for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    list_nonzero.append(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))
    

In [52]:
list_nonzero

[15, 15, 15, 15, 13, 12, 11, 10, 7, 6, 6, 6, 5, 3, 3, 2, 1, 1, 1, 1]

In [59]:
i = 0
j = len(list_nonzero)-1
while (list_nonzero[i] > max_nonzeros):
    i += 1
while (list_nonzero[j] < max_nonzeros):
    j -= 1
l1_penalty_min = l1_penalty_values[i - 1]
l1_penalty_max = l1_penalty_values[j + 1]
print 'smalled l1 penalty %s with number of nonzero more than max-non-zero index %s' % (l1_penalty_min, i-1) 
print 'largest l1 penalty %s with number of nonzero less than max-non-zero index %s' % (l1_penalty_max, j+1)

smalled l1 penalty 127.42749857 with number of nonzero more than max-non-zero index 7
largest l1 penalty 263.665089873 with number of nonzero less than max-non-zero index 9


# Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set

In [58]:
RSS_validation = []
RSS_min = float('inf')
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    nzz = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if nzz == max_nonzeros:
        prediction = model.predict(validation[all_features])
        RSS = get_RSS(prediction, validation['price'])
        if RSS < RSS_min:
            RSS_validation.append(RSS)
            RSS_min=RSS
            l1_penalty_with_lowest_rss_max_nonzero = l1_penalty
            model_with_best_rss = model
print RSS_validation
print 'best RSS for validation set in narrow range of l1 penalty', RSS_min
print 'l1_penalty with lowerst rss on validation and max non zero', l1_penalty_with_lowest_rss_max_nonzero
model_with_best_rss.coef_

[440037365263316.69]
best RSS for validation set in narrow range of l1 penalty 4.40037365263e+14
l1_penalty with lowerst rss on validation and max non zero 156.109096739


array([ -0.00000000e+00,  -0.00000000e+00,   1.06108903e+04,
         1.63380252e+02,   0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         5.06451687e+05,   4.19600436e+04,   0.00000000e+00,
         1.16253554e+05,   0.00000000e+00,   0.00000000e+00,
        -2.61223488e+03,   0.00000000e+00])